<a href="https://colab.research.google.com/github/GonzaloMartin/UTN-Diplomatura-en-Python/blob/main/Python3%20-%20Nivel%20Intermedio/Python3%20Intermedio%20-%20Unidad%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DIPLOMATURA EN PYTHON

### UNIDAD DIDÁCTICA 1

- Módulo I – Nivel Intermedio.

#### _Resueltos de [@GonzaloMartin](https://github.com/GonzaloMartin)._

## Práctica:



### Ejercicio:

Tomando en cuenta la app del nivel inicial, separarla en módulos de vista y modelo.

### Resolución:

vistas.py

In [ ]:
from tkinter import *
from tkinter.messagebox import *
from tkinter import ttk
from modelos import conexion, crear_tabla, alta, consultar, borrar, modificar, actualizar_treeview

root = Tk()
root.title("Tarea")

titulo = Label(root, text="Ingrese sus datos", bg="DarkOrchid3", fg="thistle1", height=1, width=60)
titulo.grid(row=0, column=0, columnspan=4, padx=1, pady=1, sticky=W+E)

producto = Label(root, text="Producto")
producto.grid(row=1, column=0, sticky=W)
cantidad=Label(root, text="Cantidad")
cantidad.grid(row=2, column=0, sticky=W)
precio=Label(root, text="Precio")
precio.grid(row=3, column=0, sticky=W)

a_val, b_val, c_val = StringVar(), DoubleVar(), DoubleVar()
w_ancho = 20

entrada1 = Entry(root, textvariable = a_val, width = w_ancho)
entrada1.grid(row = 1, column = 1)
entrada2 = Entry(root, textvariable = b_val, width = w_ancho)
entrada2.grid(row = 2, column = 1)
entrada3 = Entry(root, textvariable = c_val, width = w_ancho)
entrada3.grid(row = 3, column = 1)

# TreeView
tree = ttk.Treeview(root)
tree["columns"]=("col1", "col2", "col3")
tree.column("#0", width=90, minwidth=50, anchor=W)
tree.column("col1", width=112, minwidth=80)
tree.column("col2", width=112, minwidth=80)
tree.column("col3", width=112, minwidth=80)
tree.heading("#0", text="ID")
tree.heading("col1", text="Producto")
tree.heading("col2", text="Cantidad")
tree.heading("col3", text="Precio")
tree.grid(row=10, column=0, columnspan=4)

boton_alta=Button(root, text="Alta", command=lambda:alta(a_val.get(), b_val.get(), c_val.get(), tree))
boton_alta.grid(row=1, column=2, sticky=N+S+E+W)

boton_borrar=Button(root, text="Borrar", command=lambda:borrar(tree))
boton_borrar.grid(row=2, column=2, sticky=N+S+E+W)

boton_modificar = Button(root, text="Modificar", command=lambda: modificar(tree, a_val, b_val, c_val))
boton_modificar.grid(row=3, column=2, sticky=N+S+E+W)

boton_consulta=Button(root, text="Consultar", command=lambda:consultar())
boton_consulta.grid(row=6, column=2, sticky=N+S+E+W)

actualizar_treeview(tree)
root.mainloop()

modelos.py

In [ ]:
import sqlite3
import re

def conexion():
    con = sqlite3.connect("mibase.db")
    return con

def crear_tabla():
    con = conexion()
    cursor = con.cursor()

    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='productos';")
    table_exists = cursor.fetchone()

    if not table_exists:
        sql = """CREATE TABLE productos
                (id INTEGER PRIMARY KEY AUTOINCREMENT,
                producto varchar(20) NOT NULL,
                cantidad real,
                precio real)
        """

        cursor.execute(sql)
        con.commit()
        print("Tabla creada")

try:
    conexion()
    crear_tabla()
except Exception as e:
    print("Error:", e)


def alta(producto, cantidad, precio, tree):
    cadena = producto
    patron="^[A-Za-záéíóú]*$"  #regex para el campo cadena
    if(re.match(patron, cadena)):
        print(producto, cantidad, precio)
        con=conexion()
        cursor=con.cursor()
        data=(producto, cantidad, precio)
        sql="INSERT INTO productos(producto, cantidad, precio) VALUES(?, ?, ?)"
        cursor.execute(sql, data)
        con.commit()
        print("Estoy en alta todo ok")
        actualizar_treeview(tree)
    else:
        print("error en campo producto")


def consultar():
    con = conexion()
    cursor = con.cursor()
    sql = "SELECT * FROM productos ORDER BY id ASC"
    datos = cursor.execute(sql)

    print("Consulta:")
    resultado = datos.fetchall()
    for fila in resultado:
        print(fila)


def borrar(tree):
    valor = tree.selection()
    print("Borando...")
    print(valor)       #('I005',)
    item = tree.item(valor)
    print(item)        #{'text': 'I005', 'image': '', 'values': ['I005', 'Pantalon', '2', '20.0'], 'open': 0, 'tags': ''}
    print(item['text'])     #I005
    mi_id = item['text']

    con=conexion()
    cursor=con.cursor()
    data = (mi_id,)
    sql="DELETE FROM productos WHERE id = ?;"
    cursor.execute(sql, data)
    con.commit()
    tree.delete(valor)


def actualizar_treeview(mitreview):
    records = mitreview.get_children()
    for element in records:
        mitreview.delete(element)

    sql = "SELECT * FROM productos ORDER BY id ASC"
    con = conexion()
    cursor = con.cursor()
    datos = cursor.execute(sql)

    resultado = datos.fetchall()
    for fila in resultado:
        mitreview.insert('', 0, text=fila[0], values=(fila[1], fila[2], fila[3]))


def modificar(tree, producto, cantidad, precio):
    seleccion = tree.selection()

    if not seleccion:
        print("Seleccione un elemento para modificar.")
        return

    nuevo_producto = producto.get()
    nueva_cantidad = cantidad.get()
    nuevo_precio = precio.get()

    for item_id in seleccion:
        if tree.exists(item_id):
            tree.item(item_id, values=(nuevo_producto, nueva_cantidad, nuevo_precio))

            item = tree.item(item_id)
            mi_id = item['text']

            con = conexion()
            cursor = con.cursor()
            data = (nuevo_producto, nueva_cantidad, nuevo_precio, mi_id)
            sql = "UPDATE productos SET producto=?, cantidad=?, precio=? WHERE id=?"
            cursor.execute(sql, data)
            con.commit()

            print("Elemento modificado correctamente.")
        else:
            print(f"El elemento con ID {item_id} no se encuentra en el Treeview.")

### Output:

Vista del IDE con la vista y el modelo, la consola, el formulario y la base de datos:

<img src="https://raw.githubusercontent.com/GonzaloMartin/UTN-Diplomatura-en-Python/main/assets/P2-U1-Tarea1.PNG">